In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.1.1


In [2]:
import pickle
import pandas as pd

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

/home/bijay/anaconda3/envs/exp_track/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/bijay/anaconda3/envs/exp_track/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [10]:
year = 2021
month = 2

df = read_data(f'https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_{year:04d}-{month:02d}.parquet')

In [11]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [12]:
mean_predicted_duration = y_pred.mean()

In [13]:
mean_predicted_duration

16.191691679979066

In [14]:
df["ride_id"] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [15]:
df_result = pd.DataFrame()
df_result["ride_id"] = df["ride_id"]
df_result["predicted_duration"] = y_pred

In [19]:
!mkdir output

In [20]:
output_file = f"output/fhv_tripdata_{year:04d}-{month:02d}.parquet"


In [21]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)